#Financial LLM Research - Complete Pipeline




Установка зависимостей



In [ ]:
%%bash

pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
pip install -q transformers>=4.36.0 datasets>=2.15.0 accelerate>=0.25.0
pip install -q peft>=0.7.0 bitsandbytes>=0.41.0 sentencepiece protobuf
pip install -q tqdm pandas numpy scikit-learn
pip install -q rouge-score evaluate

echo "Установка завершена!"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.4 MB/s eta 0:00:00
Установка завершена!


In [ ]:
# Проверка CUDA
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("CUDA недостуен")

PyTorch version: 2.9.0+cu128
CUDA available: True
CUDA version: 12.8
GPU: Tesla T4
GPU Memory: 15.64 GB


#Создание датасета

Создадим финансовый датасет с инструкциями и ответами.

In [ ]:
import json
import random
from pathlib import Path

#Папки
Path("data").mkdir(exist_ok=True)
Path("outputs").mkdir(exist_ok=True)

print("Папки созданы")

Папки созданы


In [ ]:
# Создание датасета
def create_financial_dataset():
    """Создаёт финансовый датасет"""

    dataset = []

    # 1. Извлечение метрик
    metric_examples = [
        {
            "instruction": "Извлеки ключевые финансовые показатели из текста: Сбербанк отчитался о чистой прибыли по МСФО за 2023 год в размере 1,5 трлн рублей, что на 23% выше прошлогоднего показателя. ROE составил 24,3%, а рентабельность активов - 3,8%.",
            "output": "Ключевые показатели Сбербанка за 2023 год:\n- Чистая прибыль (МСФО): 1,5 трлн рублей\n- Рост прибыли: +23% г/г\n- ROE (рентабельность капитала): 24,3%\n- ROA (рентабельность активов): 3,8%",
            "category": "metric_extraction"
        },
        {
            "instruction": "Проанализируй финансовые метрики: Выручка компании Яндекс выросла до 365 млрд рублей (+32% г/г), EBITDA составила 89 млрд рублей с маржой 24,4%. Операционные расходы увеличились на 28%.",
            "output": "Финансовый анализ Яндекс:\n\nПоложительные факторы:\n- Выручка: 365 млрд руб (+32% г/г) - сильный рост\n- EBITDA: 89 млрд руб, маржа 24,4% - высокая операционная эффективность\n\nТочки внимания:\n- Операционные расходы +28% - рост ниже выручки, что позитивно\n- Масштабирование бизнеса с сохранением прибыльности",
            "category": "metric_analysis"
        },
    ]

    # 2. Кредитный риск
    credit_examples = [
        {
            "instruction": "Оцени кредитный риск заемщика: ИП, возраст 34 года, стаж бизнеса 5 лет, среднемесячная выручка 2,8 млн руб, чистая прибыль 420 тыс руб/мес, долговая нагрузка 15%, запрашиваемый кредит 5 млн руб на 3 года.",
            "output": "Оценка кредитного риска: НИЗКИЙ-СРЕДНИЙ\n\nПозитивные факторы:\n- Стабильный бизнес (5 лет работы)\n- Здоровая прибыльность (~15% маржа)\n- Низкая долговая нагрузка (15%)\n- Размер кредита покрывается 12-месячной прибылью\n\nРиски:\n- ИП (более высокий риск vs ООО)\n- Зависимость от одного владельца\n\nРекомендация: ОДОБРИТЬ с условиями:\n- Ставка: 12-14% годовых\n- Обеспечение: залог оборудования/недвижимости\n- Мониторинг: ежеквартальная отчетность",
            "category": "credit_risk"
        },
    ]

    # 3. ESG анализ
    esg_examples = [
        {
            "instruction": "Проанализируй ESG-риски компании: Металлургический комбинат, выбросы CO2 превышают норму на 15%, 3 трудовых спора за год, нет публичной ESG-отчетности, в составе директоров 1 женщина из 9 членов.",
            "output": "ESG-риск профиль: ВЫСОКИЙ\n\nEcological (Экология) - ВЫСОКИЙ РИСК:\n- Превышение выбросов CO2 на 15% - регуляторные риски\n- Возможные штрафы и ужесточение требований\n- Репутационные риски\n\nSocial (Социальная сфера) - СРЕДНИЙ РИСК:\n- 3 трудовых спора - индикатор проблем с персоналом\n- Риски забастовок и текучести кадров\n\nGovernance (Управление) - СРЕДНИЙ РИСК:\n- Низкое гендерное разнообразие в совете (11%)\n- Отсутствие ESG-отчетности - низкая прозрачность\n\nРекомендации:\n- Требовать план снижения выбросов\n- ESG-связанные ковенанты в кредитном договоре\n- Премия к ставке +1-2% за ESG-риски",
            "category": "esg_risk"
        },
    ]

    # 4. Анализ баланса
    statement_examples = [
        {
            "instruction": "Проанализируй баланс компании: Активы 450 млн руб (оборотные 280 млн), обязательства 320 млн (краткосрочные 180 млн), собственный капитал 130 млн. Выручка 680 млн, чистая прибыль 45 млн за год.",
            "output": "Финансовый анализ баланса:\n\n Ключевые показатели:\n- Коэффициент текущей ликвидности: 1.56 (280/180) - НОРМА\n- Коэффициент автономии: 0.29 (130/450) - НИЖЕ НОРМЫ\n- Debt-to-Equity: 2.46 (320/130) - ВЫСОКАЯ ЗАКРЕДИТОВАННОСТЬ\n- ROE: 34.6% (45/130) - ОТЛИЧНАЯ ДОХОДНОСТЬ\n- ROA: 10% (45/450) - ХОРОШО\n\n⚠️ Риски:\n- Высокая долговая нагрузка\n- Низкая доля собственного капитала\n\n✅ Преимущества:\n- Хорошая ликвидность\n- Высокая рентабельность\n\nРекомендация: Бизнес прибыльный, но требует рекапитализации для снижения финансовых рисков.",
            "category": "statement_analysis"
        },
    ]

    # 5. Анализ рынка
    market_examples = [
        {
            "instruction": "Проанализируй тренд: Ключевая ставка ЦБ повышена до 16%, инфляция 7,4%, индекс МосБиржи упал на 3,2% за неделю, рубль ослаб до 95/$.",
            "output": "Макроэкономический анализ:\n\n Негативные факторы:\n1. Ужесточение ДКП (ставка 16%) - давление на кредитование\n2. Высокая инфляция (7,4%) - риск дальнейшего повышения ставки\n3. Ослабление рубля (95/$) - импортозависимые компании под давлением\n4. Падение индекса МосБиржи (-3,2%)\n\n💼 Влияние на секторы:\n- Банки: +/- (выше маржа, но ниже спрос на кредиты)\n- Экспортеры: + (выгода от слабого рубля)\n- Ритейл: - (снижение покупательской способности)\n- Строительство: - (дорогая ипотека)\n\n📈 Прогноз:\nВозможно дальнейшее ужесточение политики ЦБ при сохранении инфляционного давления.",
            "category": "market_analysis"
        },
    ]

    # Добавляем примеры
    dataset.extend(metric_examples)
    dataset.extend(credit_examples)
    dataset.extend(esg_examples)
    dataset.extend(statement_examples)
    dataset.extend(market_examples)

    # 6. Генерация синтетических примеров (расчёты)
    for _ in range(500):
        choice = random.choice(["roe", "margin", "liquidity"])

        if choice == "roe":
            profit = random.randint(50, 500)
            equity = random.randint(200, 2000)
            roe = (profit / equity) * 100

            dataset.append({
                "instruction": f"Рассчитай ROE если прибыль {profit} млн руб, собственный капитал {equity} млн руб",
                "output": f"ROE = (Чистая прибыль / Собственный капитал) × 100% = ({profit} / {equity}) × 100% = {roe:.1f}%",
                "category": "calculation"
            })

        elif choice == "margin":
            revenue = random.randint(500, 5000)
            ebitda = random.randint(50, revenue // 2)
            margin = (ebitda / revenue) * 100

            dataset.append({
                "instruction": f"Компания имеет выручку {revenue} млн руб, EBITDA {ebitda} млн руб. Рассчитай EBITDA маржу.",
                "output": f"EBITDA маржа = (EBITDA / Выручка) × 100% = ({ebitda} / {revenue}) × 100% = {margin:.1f}%",
                "category": "calculation"
            })

        else:  # liquidity
            current_assets = random.randint(100, 1000)
            current_liab = random.randint(50, 800)
            ratio = current_assets / current_liab

            if ratio >= 2:
                assessment = "Отличная ликвидность, компания может легко покрыть краткосрочные обязательства."
            elif ratio >= 1:
                assessment = "Нормальная ликвидность, достаточно для покрытия обязательств."
            else:
                assessment = "Низкая ликвидность, риск проблем с погашением краткосрочных обязательств."

            dataset.append({
                "instruction": f"Текущие активы {current_assets} млн, краткосрочные обязательства {current_liab} млн. Оцени ликвидность.",
                "output": f"Коэффициент текущей ликвидности = {current_assets}/{current_liab} = {ratio:.2f}. {assessment}",
                "category": "ratio_analysis"
            })

    return dataset

# Создаём датасет
print("🔨 Создаём финансовый датасет...")
dataset = create_financial_dataset()
print(f"Создано {len(dataset)} примеров")

# Статистика
import pandas as pd
df = pd.DataFrame(dataset)
print("\nРаспределение по категориям:")
print(df['category'].value_counts())

Создаём финансовый датасет...
Создано 506 примеров

Распределение по категориям:
category
calculation           333
ratio_analysis        167
metric_extraction       1
metric_analysis         1
esg_risk                1
credit_risk             1
market_analysis         1
statement_analysis      1
Name: count, dtype: int64


In [ ]:
# Разделение на train/validation
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(dataset, test_size=0.1, random_state=42)

print(f"Train: {len(train_data)} примеров")
print(f"Validation: {len(val_data)} примеров")

# Сохраняем
with open('data/train.json', 'w', encoding='utf-8') as f:
    json.dump(train_data, f, ensure_ascii=False, indent=2)

with open('data/validation.json', 'w', encoding='utf-8') as f:
    json.dump(val_data, f, ensure_ascii=False, indent=2)

print("\nДатасет сохранён в data/")

Train: 455 примеров
Validation: 51 примеров

Датасет сохранён в data/


#Обучение модели с QLoRA

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset

print("Библиотеки загружены")

Библиотеки загружены


In [ ]:
# Конфигурация
MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
OUTPUT_DIR = "outputs/financial-mistral-qlora"

# BitsAndBytes config (4-bit quantization)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

print("Конфигурация готова")

Конфигурация готова


In [ ]:

# ЗАГРУЗКА БАЗОВОЙ МОДЕЛИ MISTRAL (БЕЗ АДАПТЕРОВ)

import gc
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Очистка памяти
gc.collect()
torch.cuda.empty_cache()

print("Загружаем БАЗОВУЮ модель Mistral-7B...")

# ИСПОЛЬЗУЕМ БАЗОВУЮ МОДЕЛЬ (без LoRA адаптеров!)
MODEL_NAME = "mistralai/Mistral-7B-v0.1"

# Конфигурация квантизации
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Загрузка модели
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
    use_cache=False,
    low_cpu_mem_usage=True,
)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

print("Модель загружена успешно!")
print(f"Память GPU: {torch.cuda.memory_allocated()/1e9:.2f} GB")
print(f"Vocab size: {len(tokenizer)}")

Загружаем БАЗОВУЮ модель Mistral-7B...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Модель загружена успешно!
Память GPU: 4.13 GB
Vocab size: 32000


In [ ]:
# Подготовка модели для обучения
model.config.use_cache = False
model.config.pretraining_tp = 1
model = prepare_model_for_kbit_training(model)

# LoRA конфигурация
lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

# Статистика
trainable_params = 0
all_params = 0
for _, param in model.named_parameters():
    all_params += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()

print("\nПараметры модели:")
print(f"Всего: {all_params:,}")
print(f"Обучаемых: {trainable_params:,}")
print(f"роцент: {100 * trainable_params / all_params:.4f}%")


Параметры модели:
Всего: 3,919,843,328
Обучаемых: 167,772,160
роцент: 4.2801%


In [ ]:
# Загрузка датасета
dataset = load_dataset(
    "json",
    data_files={
        "train": "data/train.json",
        "validation": "data/validation.json",
    }
)

print(f"Датасет загружен")
print(f"Train: {len(dataset['train'])}")
print(f"Validation: {len(dataset['validation'])}")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Датасет загружен
Train: 455
Validation: 51


In [ ]:
# Токенизация
def tokenize_function(examples):


    full_texts = []

    for instruction, output in zip(examples['instruction'], examples['output']):
        # Объединяем всё в один текст
        full_text = f"[INST] {instruction} [/INST] {output}</s>"
        full_texts.append(full_text)

    # Токенизируем весь текст целиком
    model_inputs = tokenizer(
        full_texts,
        max_length=2048,
        padding=False,
        truncation=True,
        return_tensors=None,
    )


    model_inputs["labels"] = model_inputs["input_ids"].copy()

    return model_inputs

print("Токенизация датасета...")
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
)
print("Токенизация завершена")

Токенизация датасета...


Map:   0%|          | 0/455 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

Токенизация завершена


In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    learning_rate=2e-4,
    weight_decay=0.01,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    max_grad_norm=0.3,
    bf16=True,
    logging_steps=10,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=3,
    eval_strategy="steps",  # ИЗМЕНИЛИ: evaluation_strategy → eval_strategy
    eval_steps=100,
    load_best_model_at_end=True,
    report_to="none",
    group_by_length=True,
    remove_unused_columns=False,
)

print("Training arguments готовы")

warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


Training arguments готовы


In [ ]:
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
)

print("Trainer готов к обучению")

Trainer готов к обучению


In [ ]:
# ОБУЧЕНИЕ
print("\n" + "="*60)
print("НАЧИНАЕМ ОБУЧЕНИЕ")
print("="*60)
print(f"Эпох: {training_args.num_train_epochs}")
print(f"Эффективный batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"GPU память: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

print("="*60 + "\n")

# Запускаем бучение
trainer.train()

print("\n" + "="*60)
print("ОБУЧЕНИЕ ЗАВЕРШЕНО!")
print("="*60)


НАЧИНАЕМ ОБУЧЕНИЕ
Эпох: 3
Эффективный batch size: 16
GPU память: 5.33 GB



Step,Training Loss,Validation Loss



ОБУЧЕНИЕ ЗАВЕРШЕНО!


In [ ]:
# Сохранение модели
final_model_path = f"{OUTPUT_DIR}/final_model"
trainer.model.save_pretrained(final_model_path)
tokenizer.save_pretrained(final_model_path)

print(f"Модель сохранена в {final_model_path}")

Модель сохранена в outputs/financial-mistral-qlora/final_model


#Оценка модели

Проверим качество обученной модели.

Перед этим лучше перезапустить среду выполнения

In [ ]:
OUTPUT_DIR = "outputs/financial-mistral-qlora"
final_model_path = f"{OUTPUT_DIR}/final_model"

In [ ]:
import gc
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

# Очистка
gc.collect()
torch.cuda.empty_cache()

print("Загружаем модель для инференса...")

MODEL_NAME = "mistralai/Mistral-7B-v0.1"
final_model_path = "outputs/financial-mistral-qlora/final_model"


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)


base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map={"": 0},
    torch_dtype=torch.float16,
)


inference_model = PeftModel.from_pretrained(
    base_model,
    final_model_path,
    device_map={"": 0},
)


inference_model = inference_model.to("cuda")


inference_model.eval()


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"Модель загружена и на GPU!")
print(f"Устройство: {next(inference_model.parameters()).device}")
print(f"Память: {torch.cuda.memory_allocated()/1e9:.2f} GB")

Загружаем модель для инференса...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

Модель загружена и на GPU!
Устройство: cuda:0
Память: 4.81 GB


In [ ]:
# Функция для генерации

def generate_response(instruction, max_new_tokens=512):


    prompt = f"[INST] {instruction} [/INST]"


    inputs = tokenizer(prompt, return_tensors="pt")


    input_ids = inputs["input_ids"].to("cuda")
    attention_mask = inputs["attention_mask"].to("cuda")


    with torch.no_grad():
        outputs = inference_model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )


    response = tokenizer.decode(outputs[0], skip_special_tokens=True)


    if "[/INST]" in response:
        response = response.split("[/INST]")[-1].strip()

    return response



print("Функция генерации готова")

Функция генерации готова


In [ ]:
print("Проверка устройств:")
print(f"Модель на: {inference_model.device}")
print(f"CUDA доступна: {torch.cuda.is_available()}")


if str(inference_model.device) == "cpu":

    inference_model = inference_model.to("cuda")
    print("Перенесено на GPU")

Проверка устройств:
Модель на: cuda:0
CUDA доступна: True


In [ ]:
# Тестовые примеры
test_examples = [
    "Рассчитай ROE если прибыль 150 млн руб, капитал 600 млн руб",
    "Текущие активы 500 млн, краткосрочные обязательства 300 млн. Оцени ликвидность.",
    "Компания имеет выручку 1000 млн, EBITDA 250 млн. Рассчитай EBITDA маржу.",
]

print("\n" + "="*80)
print("ТЕСТИРОВАНИЕ МОДЕЛИ")
print("="*80 + "\n")

for i, example in enumerate(test_examples, 1):
    print(f"\n[Пример {i}/{len(test_examples)}]")
    print("─" * 80)
    print(f"Вопрос: {example}")
    print(f"\nОтвет модели:")
    response = generate_response(example)
    print(response)
    print("\n" + "─" * 80)


ТЕСТИРОВАНИЕ МОДЕЛИ


[Пример 1/3]
────────────────────────────────────────────────────────────────────────────────
Вопрос: Рассчитай ROE если прибыль 150 млн руб, капитал 600 млн руб

Ответ модели:
ROE = (Чистая прибыль / Собственный капитал) × 100% = (150 / 600) × 100% = 25.0%

────────────────────────────────────────────────────────────────────────────────

[Пример 2/3]
────────────────────────────────────────────────────────────────────────────────
Вопрос: Текущие активы 500 млн, краткосрочные обязательства 300 млн. Оцени ликвидность.

Ответ модели:
Коэффициент текущей ликвидности = 500/300 = 1.67. Нормальная ликвидность, достаточно для покрытия обязательств.

────────────────────────────────────────────────────────────────────────────────

[Пример 3/3]
────────────────────────────────────────────────────────────────────────────────
Вопрос: Компания имеет выручку 1000 млн, EBITDA 250 млн. Рассчитай EBITDA маржу.

Ответ модели:
EBITDA маржа = (EBITDA / Выручка) × 100% = (250 / 1000

In [ ]:
import random

In [ ]:
# Простая оценка на валидационной выборке
import json
from tqdm.auto import tqdm

# Загружаем валидационные данные
with open('data/validation.json', 'r', encoding='utf-8') as f:
    val_data = json.load(f)

# Берём 10 случайных примеров для быстрой оценки
sample_val = random.sample(val_data, min(10, len(val_data)))

print("\nОценка на валидационной выборке (10 примеров):")
print("="*80 + "\n")

predictions = []
references = []

for example in tqdm(sample_val, desc="Генерация"):
    pred = generate_response(example['instruction'])
    predictions.append(pred)
    references.append(example['output'])

print("\nГенерация завершена")


Оценка на валидационной выборке (10 примеров):



Генерация:   0%|          | 0/10 [00:00<?, ?it/s]


Генерация завершена


In [ ]:
# Вычисление метрик
from rouge_score import rouge_scorer
import numpy as np

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for pred, ref in zip(predictions, references):
    scores = scorer.score(ref, pred)
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)

print("\nРЕЗУЛЬТАТЫ ОЦЕНКИ:")
print("="*80)
print(f"ROUGE-1: {np.mean(rouge1_scores):.4f}")
print(f"ROUGE-2: {np.mean(rouge2_scores):.4f}")
print(f"ROUGE-L: {np.mean(rougeL_scores):.4f}")
print("="*80)


РЕЗУЛЬТАТЫ ОЦЕНКИ:
ROUGE-1: 0.8268
ROUGE-2: 0.7857
ROUGE-L: 0.8268


#Интерактивное тестирование



In [ ]:

your_question = "Оцени кредитный риск: ИП, стаж 2 года, выручка 1 млн/мес, долг 30%"  # ИЗМЕНИ ЭТО

print("\nВопрос:")
print(your_question)
print("\nОтвет модели:")
print("─" * 80)
response = generate_response(your_question)
print(response)
print("─" * 80)


Вопрос:
Оцени кредитный риск: ИП, стаж 2 года, выручка 1 млн/мес, долг 30%

Ответ модели:
────────────────────────────────────────────────────────────────────────────────
Коэффициент текущей ликвидности = 300/1000 = 0.30. ⚠️ Низкая ликвидность, риск проблем с погашением краткосрочных обязательств.
────────────────────────────────────────────────────────────────────────────────


#Сохранение результатов

In [ ]:
import json
import numpy as np

results = {
    "model": MODEL_NAME,
    "dataset_size": len(val_data),  # Используем val_data вместо dataset
    "val_samples": len(val_data),
    "epochs": 3,
    "metrics": {
        "rouge1": float(np.mean(rouge1_scores)),
        "rouge2": float(np.mean(rouge2_scores)),
        "rougeL": float(np.mean(rougeL_scores)),
    },
    "sample_predictions": [
        {
            "instruction": sample_val[i]['instruction'],
            "reference": sample_val[i]['output'],
            "prediction": predictions[i]
        }
        for i in range(min(3, len(predictions)))
    ]
}

# Сохраняем
OUTPUT_DIR = "outputs/financial-mistral-qlora"

with open(f'{OUTPUT_DIR}/results.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"Результаты сохранены в {OUTPUT_DIR}/results.json")


print("\n" + "="*80)
print("ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ")
print("="*80)
print(f"\nМодель: {MODEL_NAME}")
print(f"Эпох обучения: 3")
print(f"\nМетрики качества:")
print(f"  ROUGE-1: {results['metrics']['rouge1']:.4f}")
print(f"  ROUGE-2: {results['metrics']['rouge2']:.4f}")
print(f"  ROUGE-L: {results['metrics']['rougeL']:.4f}")
print("\n" + "="*80)

Результаты сохранены в outputs/financial-mistral-qlora/results.json

ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ

Модель: mistralai/Mistral-7B-v0.1
Эпох обучения: 3

Метрики качества:
  ROUGE-1: 0.8268
  ROUGE-2: 0.7857
  ROUGE-L: 0.8268

